In [98]:
import numpy as np
import pandas as pd
import re
from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [3]:
# load csv

df = pd.read_csv("./CandidateSummaryAction1.csv")
df

,can_id,can_nam,can_off,can_off_sta,can_off_dis,can_par_aff,can_inc_cha_ope_sea,can_str1,can_str2,can_cit,...,cas_on_han_beg_of_per,cas_on_han_clo_of_per,net_con,net_ope_exp,deb_owe_by_com,deb_owe_to_com,cov_sta_dat,cov_end_dat,winner,votes
0,H2GA12121,"ALLEN, RICHARD W",H,GA,12.0,REP,INCUMBENT,2237 PICKENS RD,NaN,AUGUSTA,...,"$60,109.57","$175,613.35","$1,074,949.50","$907,156.21","$1,125,000.00",NaN,1/1/2015,10/19/2016,Y,158708.0
1,H6PA02171,"EVANS, DWIGHT",H,PA,2.0,DEM,CHALLENGER,PO BOX 6578,NaN,PHILADELPHIA,...,NaN,"$105,687.23","$1,406,719.06","$1,298,831.83",NaN,NaN,11/2/2015,10/19/2016,Y,310770.0
2,H6FL04105,"RUTHERFORD, JOHN",H,FL,4.0,REP,OPEN,3817 VICKERS LAKE DRIVE,NaN,JACKSONVILLE,...,NaN,"$35,645.09","$650,855.38","$656,210.29","$60,000.00",NaN,4/1/2016,10/19/2016,Y,286018.0
3,H4MT01041,"ZINKE, RYAN K",H,MT,0.0,REP,INCUMBENT,409 2ND ST W,NaN,WHITEFISH,...,"$83,815.78","$399,122.61","$4,938,943.74","$5,055,942.15",NaN,NaN,1/1/2015,10/19/2016,Y,280476.0
4,H8CA09060,"LEE, BARBARA",H,CA,13.0,DEM,INCUMBENT,"409 13TH ST, 17TH FL",NaN,OAKLAND,...,"$83,690.60","$181,338.23","$1,197,676.61","$949,488.98","$4,401.75",NaN,1/1/2015,10/19/2016,Y,277390.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,P60005931,"BUTTOCKS, SYDNEYS VOLUPTOUS",P,US,0.0,IND,OPEN,143 HIGHLAND AVENUE,NaN,BUFFALO,...,"$5,000.00","$5,000.00",NaN,NaN,NaN,NaN,3/3/2015,3/31/2015,NaN,NaN
1810,S6CA00568,"MOLNAR, ROBERT",S,CA,0.0,REP,OPEN,PO BOX 321231,NaN,LOS GATOS,...,"$1,000.00",NaN,"$1,000.00","$1,000.00",NaN,NaN,1/1/2015,3/31/2015,NaN,NaN
1811,S6MD03524,"HOLMES, GREGORY",S,MD,0.0,REP,OPEN,13601 MACDERNUT CT,NaN,UPPER MARLBORO,...,NaN,NaN,NaN,NaN,NaN,NaN,8/29/2017,2/1/2015,NaN,NaN
1812,H6AS00031,"HUNKIN-FINAU, VAITINASA SALU",H,AS,NaN,DEM,NaN,PO BOX 252 VAILOAUTA,NaN,PAGO PAGO,...,"$1,058.44","$4,561.00","$41,670.00","$38,167.44",NaN,NaN,7/1/2016,10/19/2016,NaN,NaN


In [ ]:
# we want to remove anyone who got 0 votes - also want to look for more tightly contested races as our question
# doesn't really pertain to unopposed or landslide campaigns

# aggregate based on candidate_incumbent_challenger_open_seat as one
# type of office cand_office for house/senate/pres
# can also group spending by states cand_office_st
# cand_office_dist might be interesting to see spending across a state

# type of contributions - can compare individual_contribution
# other_committee_contribution - contributions from pacs/other candidates
# cand_contribution - from self
# party_committee_contribution - from state/national party committees
# total contribution
# coverage_start_date - when a candidate started reporting finances note- dates are from 1/1/2015 - 10/19/2016
# winner column represents if that candidate won - can be represented with boolean
# is there a correlation between spending and cook partisan index(do this later)

In [13]:
df.columns

Index(['can_id', 'can_nam', 'can_off', 'can_off_sta', 'can_off_dis',
       'can_par_aff', 'can_inc_cha_ope_sea', 'can_str1', 'can_str2', 'can_cit',
       'can_sta', 'can_zip', 'ind_ite_con', 'ind_uni_con', 'ind_con',
       'par_com_con', 'oth_com_con', 'can_con', 'tot_con',
       'tra_fro_oth_aut_com', 'can_loa', 'oth_loa', 'tot_loa',
       'off_to_ope_exp', 'off_to_fun', 'off_to_leg_acc', 'oth_rec', 'tot_rec',
       'ope_exp', 'exe_leg_acc_dis', 'fun_dis', 'tra_to_oth_aut_com',
       'can_loa_rep', 'oth_loa_rep', 'tot_loa_rep', 'ind_ref', 'par_com_ref',
       'oth_com_ref', 'tot_con_ref', 'oth_dis', 'tot_dis',
       'cas_on_han_beg_of_per', 'cas_on_han_clo_of_per', 'net_con',
       'net_ope_exp', 'deb_owe_by_com', 'deb_owe_to_com', 'cov_sta_dat',
       'cov_end_dat', 'winner', 'votes'],
      dtype='object')

In [77]:
parsed_df = df[['can_nam', 'can_off', 'can_off_sta', 'can_off_dis', 'can_par_aff', 'can_inc_cha_ope_sea', 'ind_con', 'can_con', 'tot_con', 'winner', 'votes']]

In [79]:
parsed_df.votes.map(lambda x: )

0       158708.0
1       310770.0
2       286018.0
3       280476.0
4       277390.0
          ...   
1809         NaN
1810         NaN
1811         NaN
1812         NaN
1813         NaN
Name: votes, Length: 1814, dtype: float64

In [92]:

fill_values = {"can_con": 0.0, "winner": "N", "votes": 0.0}
parsed_df = parsed_df.fillna(value=fill_values)
parsed_df = parsed_df[parsed_df['votes'] != 0.0]

In [93]:
parsed_df

,can_nam,can_off,can_off_sta,can_off_dis,can_par_aff,can_inc_cha_ope_sea,ind_con,can_con,tot_con,winner,votes
0,"ALLEN, RICHARD W",H,GA,12.0,REP,INCUMBENT,"$601,274.50",0.0,"$1,074,949.50",Y,158708.0
1,"EVANS, DWIGHT",H,PA,2.0,DEM,CHALLENGER,"$1,114,711.02",0.0,"$1,417,545.22",Y,310770.0
2,"RUTHERFORD, JOHN",H,FL,4.0,REP,OPEN,"$542,105.38","$2,700.00","$650,855.38",Y,286018.0
3,"ZINKE, RYAN K",H,MT,0.0,REP,INCUMBENT,"$4,317,331.58",0.0,"$4,980,915.41",Y,280476.0
4,"LEE, BARBARA",H,CA,13.0,DEM,INCUMBENT,"$897,123.61",0.0,"$1,205,863.61",Y,277390.0
...,...,...,...,...,...,...,...,...,...,...,...
374,"ANGELLE, SCOTT MR.",H,LA,3.0,REP,CHALLENGER,"$958,381.91",0.0,"$1,100,481.91",Y,91374.0
375,"GALLEGO, RUBEN",H,AZ,7.0,DEM,INCUMBENT,"$544,837.00",0.0,"$1,025,225.14",Y,88219.0
376,"JOHNSON, JAMES MICHAEL",H,LA,4.0,REP,OPEN,"$527,766.70","$1,063.28","$556,029.98",Y,70556.0
377,"BARRAGAN, NANETTE",H,CA,44.0,DEM,OPEN,"$1,276,552.70","$25,167.92","$1,467,446.28",Y,70303.0


In [101]:
re.sub('[^0-9\.]', '', parsed_df['ind_con'][0])

'601274.50'

In [112]:
parsed_df['ind_con'] = parsed_df.ind_con.map(lambda x: float(re.sub('[^0-9\.]', '', str(x))))

In [113]:
parsed_df['can_con'] = parsed_df.can_con.map(lambda x: float(re.sub('[^0-9\.]', '', str(x))))